In [23]:
import vcf

def vcf2dict(loc):
    """
    For a vcf file returns dict with all its records.

    The Key:Value pair is structured as {'position':'full vcf record object'}
    :param loc:string
    :return:dict{int:obj}
    """
    reader = vcf.Reader(open(loc, 'r'))
    records = {}
    for record in reader:
        records[record.POS] = record
    return records

def vcf_isec(vcf1, vcf2):
    """
    For 2 vcf dicts structured as {'position':'full vcf record object'}, returns the intersection of variants in a similarly structured dict.
    :param vcf1: dict{int:obj}
    :param vcf2: dict{int:obj}
    :return: dict{int:obj}
    """
    isec_records = {}
    for record in vcf1.keys():
        #https://stackoverflow.com/questions/3733992/determine-whether-a-key-is-present-in-a-dictionary
        if record in vcf2:
            isec_records[record]=vcf2[record]
    # Logic could be added to check if dict is empty or not as in:
    # https://stackoverflow.com/questions/23177439/python-checking-if-a-dictionary-is-empty-doesnt-seem-to-work
    # But for modularity this list will just return and empty dict
    return isec_records

# Import the Portuguese (ERR4157959) and Spanish (ERR4395294) sample
pt_records = vcf2dict('D:/Bioinformatica/Laboratorios/trabalho/variantes/pt.raw.vcf')
es_records = vcf2dict('D:/Bioinformatica/Laboratorios/trabalho/variantes/es.raw.vcf')


# Find intersection
pt_es_isec = vcf_isec(pt_records,es_records)

# Produce human readable answer (checks if there is an isec or not)
if pt_es_isec:
    print("Existem variantes presentes nas duas amostras, estes estão localizados em {0}".format(pt_es_isec.keys()))
else:
    print("Não existem variantes repetidos nas duas amostras.")

Não existem variantes repetidos nas duas amostras.


Recorremos ao seguinte site para saber o id do ensembl do nosso gene de interesse (N):
https://covid-19.ensembl.org/Sars_cov_2/Gene/Summary?db=core;g=ENSSASG00005000005;r=MN908947.3:28274-29533;t=ENSSAST00005000005

Os resultados foram filtrados para apresentar apenas o gene de interesse: ENSSASG00005000005

In [33]:
from BCBio import GFF
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqFeature
from collections import defaultdict


def gff2rec(in_file):
    """
    For a gff or gtf file returns list with all its genes.
    :param in_file:string
    :return:list[object]
    """
    in_handle = open(in_file)
    # https://biopython.org/wiki/GFF_Parsing
    # This method is not good for massive gff files, read the doc
    records = []
    for record in GFF.parse(in_handle):
        # This is dirty Python
        records.append(record)
    in_handle.close()
    return records

def seq_rec_sgenes(record):
    """
    For SeqRecord returns list with all its genes that might have been studied for function.
    :param record:string
    :return:list[object]
    """
    studied_genes = []
    for feature in record.features:
        if feature.type == "gene" and feature.qualifiers['gene_name'] and feature.qualifiers['gene_biotype']:
            studied_genes.append(feature)
    return studied_genes

def lookup_var2genes(variations, record):
    """
    For a list of SeqFeature objects, it performs a lookup of variations provided in a dict structured as {'position':'pyvcf record object'}.

    This operation returns a dict where for gene there is a list of variation positions.
    :param variations: dict{int:obj}
    :param record: list[object]
    :return: dict{string:list[int]}
    """
    genes_var = defaultdict(list)
    for gene in seq_rec_sgenes(record):
        # The "gene_id" qualifier is a list of len 1 so I have to call the index 0 otherwise in futher opperations I will try to add a list as a dict key.
        # This raises an error as list are not hashable
        gene_id = gene.qualifiers['gene_id'][0]
        genes_var[gene_id] = []
        for var in variations.keys():
            # Variant position
            # Gene location using the logic in:
            # https://biopython.readthedocs.io/en/latest/api/Bio.SeqFeature.html#Bio.SeqFeature.FeatureLocation
            if gene.location.start < variations[var].POS < gene.location.end:
                # If the var is within the start and end of the gene, it means it belongs to it
                # how to add to lists inside dict:
                # https://stackoverflow.com/questions/26367812/appending-to-list-in-python-dictionary
                genes_var[gene_id].append(var)
    return genes_var

# Import SarsCov2 ref seq genes
SarsCov2_file = "D:/Bioinformatica/Laboratorios/trabalho/variantes/Sars_cov_2.ASM985889v3.101.gtf"
sars_cov_rec = gff2rec(SarsCov2_file)[0]


genes_with_var_es = lookup_var2genes(es_records, sars_cov_rec)
genes_with_var_pt = lookup_var2genes(pt_records, sars_cov_rec)


# Print the gene variations per gene
for gene in genes_with_var_es:
    if genes_with_var_es[gene]:
        if gene == 'ENSSASG00005000005':
            # Dont print gene with no
            print("The spanish sample has detected variations in the positions:\n{0} for the following studied gene {1}\n".format(genes_with_var_es[gene], gene))
for gene in genes_with_var_pt:
    if genes_with_var_pt[gene]:
        if gene == 'ENSSASG00005000005':
            # Dont print gene with no
            print("The portuguese sample has detected variations in the positions:\n{0} for the following studied gene {1}".format(genes_with_var_pt[gene], gene))

The spanish sample has detected variations in the positions:
[28657, 28863] for the following studied gene ENSSASG00005000005

The portuguese sample has detected variations in the positions:
[28881, 28882, 28883] for the following studied gene ENSSASG00005000005
